In [ ]:
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#
#from selenium.webdriver.common.action_chains import ActionChains
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#import pickle
#
#import os
#import time
#import pandas as pd
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#
#from selenium import webdriver
#from selenium.webdriver.firefox.options import Options
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#import time
#import pandas as pd
#
#from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

In [1]:
import pandas as pd
import os
import pickle
import time
import json
from collections import defaultdict
from openai import OpenAI
from datetime import datetime
import subprocess

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")


In [4]:

#https://www.youtube.com/watch?v=dCHh2HacOYk
# Trump MELTDOWN on Walmart + Saudi news
# 0UtHedAXIkA
# Indians Finally Admit Pakistan Won

# https://www.youtube.com/watch?v=xJ_omX4Ejqw
# India Beat Pakistan's Air Defense

# https://www.youtube.com/watch?v=7tqYgfhCwb4&t=154s&pp=ygUOc21pbGUgMiBqYW5uYWg%3D
# Indians Declare WAR on Aamir Khan


### below is to run a loop and get all

In [9]:



# Load your JSON file
with open(f"comments_scraped/{video_id}_{timestamp}.info.json", "r", encoding="utf-8") as f:
    all_data=json.load(f)        
    raw_comments = all_data['comments']
    title=all_data['title']
    uploader_id=all_data['uploader_id']

print(title,uploader_id)



Joe Rogan Mocks Pro-Israeli Douglas Murray @smiletojannah


In [ ]:
# Group comments by ID and organize replies under their parent
comment_dict = {}
replies_grouped = defaultdict(list)

# First pass to separate top-level and replies
for comment in raw_comments:
    print(comment)
    if comment.get("parent") and comment["parent"] != "root":
        replies_grouped[comment["parent"]].append({
            "id": comment["id"],
            "text": comment["text"],
            "author": comment["author"],
            "likes": comment["like_count"],
            "timestamp": comment["timestamp"]
        })
    else:
        comment_dict[comment["id"]] = {
            "id": comment["id"],
            "text": comment["text"],
            "author": comment["author"],
            "likes": comment["like_count"],
            "timestamp": comment["timestamp"],
            "replies": []
        }

# Second pass to attach replies to top-level comments
for parent_id, replies in replies_grouped.items():
    if parent_id in comment_dict:
        comment_dict[parent_id]["replies"] = replies

# Convert dict to list
structured_comments = list(comment_dict.values())

# Print a few samples
for comment in structured_comments[:3]:  # limit to 3 for brevity
    print(f"Comment by {comment['author']}: {comment['text']}")
    for reply in comment["replies"]:
        print(f"  ↳ Reply by {reply['author']}: {reply['text']}")
    print("-" * 60)

In [ ]:
base_filename=f'{video_id}'
rows = []
for comment in structured_comments:
    rows.append({
        "id": comment["id"],
        "text": comment["text"],
        "author": comment["author"],
        "likes": comment["likes"],
        "timestamp": comment["timestamp"],
        "is_reply": False,
        "parent_id": None
    })
    for reply in comment.get("replies", []):
        rows.append({
            "id": reply["id"],
            "text": reply["text"],
            "author": reply["author"],
            "likes": reply["likes"],
            "timestamp": reply["timestamp"],
            "is_reply": True,
            "parent_id": comment["id"]
        })

# Create a DataFrame
df = pd.DataFrame(rows)



In [ ]:
df

In [ ]:
df.sort_values(by=['likes'],inplace=True,ascending=False)

In [ ]:
# Export to CSV
df.to_csv(f"{base_filename}.csv", index=False)

# Export to Pickle
with open(f"{base_filename}.pkl", "wb") as f:
    pickle.dump(structured_comments, f)

print("Export completed: CSV and PKL saved.")


In [ ]:
df_comment1=df[df['is_reply']==False]

In [ ]:
df_comment1

In [ ]:
comment1_list=list(df_comment1['text'])

In [ ]:
print(comment1_list[:2])

In [ ]:
k=5
topkcomments_list=comment1_list[:int(0.05*len(comment1_list))]

In [ ]:
len(topkcomments_list),len(comment1_list)

In [ ]:
topkcomments_str='\n'.join(topkcomments_list)

In [ ]:
api_key='sk-bedceae2ceba437f944db22706354095'

In [ ]:
client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com",
)

system_prompt = """
The user will provide a list of YouTube comments. Please analyze all the comments together and generate a single, structured JSON object summarizing the overall qualitative dynamics.

EXAMPLE INPUT:
["Great job! This video really opened my eyes.", "What a biased take. Shameful.", "😂😂 you're so clueless it's funny.", "Pakistan zindabad!", "Link to giveaway 👉 http://spamlink"]

EXAMPLE JSON OUTPUT:
{
  "overall_sentiment_distribution": {
    "positive": 1,
    "neutral": 1,
    "negative": 3
  },
  "dominant_emotions": ["anger", "sarcasm", "joy"],
  "toxic_comment_count": 2,
  "controversy_score": 0.75,
  "key_topics": ["bias in media", "nationalism", "truth and misinformation"],
  "frequent_bias_or_group_mentions": ["Pakistan", "India", "YouTube creators"],
  "sarcasm_detected": true,
  "languages_detected": ["English", "Urdu"],
  "spam_comment_count": 1,
  "summary": "The comment section is emotionally charged with a mix of national pride, strong criticism, and sarcasm. There's significant polarization, and a moderate amount of toxicity and spam. These patterns suggest the topic is controversial and has drawn viewers from diverse backgrounds with opposing views."
}

Please return the analysis in this format. Focus on producing insightful, unbiased feedback that helps a content creator understand audience sentiment, engagement, controversy, and risks.
"""

user_prompt = topkcomments_str

messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    response_format={
        'type': 'json_object'
    }
)

print(json.loads(response.choices[0].message.content))

In [ ]:
llm_analysis=json.loads(response.choices[0].message.content)
llm_analysis['title']=title
llm_analysis['uploader_id']=uploader_id



In [ ]:

# Construct the filename
filename = f"insights/{video_id}_insight1_{timestamp}.json"
# Save to JSON file
with open(filename, "w", encoding="utf-8") as f:
    json.dump(llm_analysis, f, ensure_ascii=False, indent=2)
print(f"Saved analysis to {filename}")    